# Set up environment

In [1]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

⛳️ Local environment
Adding the following directory to the PYTHONPATH: /Users/annysiadupaya/Documents/code/art-recommendations


# <span style="color:#ff5f27"> Scheduling Hopsworks materialization jobs </span>
## 📝 Imports

In [2]:
from datetime import datetime, timezone

from recsys import hopsworks_integration

/Users/annysiadupaya/Documents/code/art-recommendations/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [3]:
project, fs = hopsworks_integration.get_feature_store()

jobs_api = project.get_jobs_api()

2025-01-01 01:20:00.468 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:15 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2025-01-01 01:20:00,471 INFO: Initializing external client
2025-01-01 01:20:00,472 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-01 01:20:01,807 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1197208


# Retrieving materialization jobs


In [4]:
interactions_job = jobs_api.get_job('interactions_1_offline_fg_materialization')
interactions_job

Job('interactions_1_offline_fg_materialization', 'SPARK')

In [5]:
transactions_job = jobs_api.get_job('transactions_1_offline_fg_materialization')
transactions_job

Job('transactions_1_offline_fg_materialization', 'SPARK')

# Running materialization jobs


In [6]:
interactions_job_execution = interactions_job.run()
interactions_job_execution

Launching job: interactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1197208/jobs/named/interactions_1_offline_fg_materialization/executions
2025-01-01 01:20:48,905 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-01 01:20:52,075 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-01 01:21:55,433 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-01-01 01:21:55,601 INFO: Waiting for log aggregation to finish.
2025-01-01 01:22:04,149 INFO: Execution finished successfully.


Execution('SUCCEEDED', 'FINISHED', '2025-01-01T00:20:34.000Z', '-op offline_fg_materialization -path hdfs:///Projects/id2223artsy/Resources/jobs/interactions_1_offline_fg_materialization/config_1735506641958')

In [7]:
transactions_job_execution = transactions_job.run()
transactions_job_execution

Launching job: transactions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1197208/jobs/named/transactions_1_offline_fg_materialization/executions
2025-01-01 01:22:16,304 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-01-01 01:22:19,470 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-01 01:23:22,884 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-01-01 01:23:23,039 INFO: Waiting for log aggregation to finish.
2025-01-01 01:23:31,648 INFO: Execution finished successfully.


Execution('SUCCEEDED', 'FINISHED', '2025-01-01T00:22:08.000Z', '-op offline_fg_materialization -path hdfs:///Projects/id2223artsy/Resources/jobs/transactions_1_offline_fg_materialization/config_1735506503029')

## <span style="color:#ff5f27">⏰ Scheduling materialization jobs </span>


In [8]:
interactions_job.schedule(
    cron_expression="0 0 0 * * ?",  # Runs at midnight (00:00:00) every day
    start_time=datetime.now(tz=timezone.utc),
)
interactions_job.job_schedule.next_execution_date_time

datetime.datetime(2025, 1, 2, 0, 0, tzinfo=datetime.timezone.utc)

In [9]:
transactions_job.schedule(
    cron_expression="0 0 0 * * ?",  # Runs at midnight (00:00:00) every day
    start_time=datetime.now(tz=timezone.utc),
)
transactions_job.job_schedule.next_execution_date_time

datetime.datetime(2025, 1, 2, 0, 0, tzinfo=datetime.timezone.utc)